## EDA FOR THE CLEANED DATASET

This brief notebook will perform some of the various EDA techniques used on the raw dataset in order to understand not only if the cleaned dataset is complete and suitable for further preprocessing-modeling steps, but also to see if the shape of the data changes importantly, or not. 

In [1]:
file_path = r'C:\unibo-dtm-ml-2526-cervical-cancer-predictor\data\cleaned_data.csv'
with open(file_path, 'r') as f:
    lines = f.readlines()
print('Read {} lines'.format(len(lines)))

Read 836 lines


Importing the necessary libraries. 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


Outlining the renewed profile of the dataset. 

In [3]:
# repeat the data profiling pipeline for the newly cleaned data
df = pd.read_csv(file_path)

print("\nDataset Info: \n")
print(df.info())

#check whether everything went smoothly at the data cleaning stage
print("\nMissing Values: \n")
print(df.isnull().sum()) 

print("\nDuplicate Values: \n")
print(df.duplicated().sum())

print("\nDescriptive Statistics:")
print(df.describe(include='all'))



Dataset Info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 835 entries, 0 to 834
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              835 non-null    int64  
 1   Number of sexual partners        810 non-null    float64
 2   First sexual intercourse         828 non-null    float64
 3   Num of pregnancies               779 non-null    float64
 4   Smokes (years)                   822 non-null    float64
 5   Smokes (packs/year)              822 non-null    float64
 6   Hormonal Contraceptives (years)  732 non-null    float64
 7   IUD (years)                      723 non-null    float64
 8   STDs (number)                    735 non-null    float64
 9   STDs: Viral group                835 non-null    int64  
 10  STDs: Bacterial group            835 non-null    int64  
 11  STDs:condylomatosis              735 non-null    float64
 12  STDs: